In [14]:
import torch
from torch.nn import functional as F
from torch import nn

d_model = 12
block_size = 6
batch_size = 16
n_heads = 2
dropout = 0.1
learning_rate = 1e-3
epochs = 10_000
eval_iters = 200

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True

In [15]:
# Multi-head attention class

class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, d_model,  dropout=0.1):

        super().__init__()
        assert d_model % n_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.n_heads = n_heads
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.key = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)
        self.att_proj = nn.Linear(d_model, d_model, bias=False)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size), diagonal=1).bool())

    def forward(self, x):

        q = x
        k = x
        v = x
        B,T,_ = x.shape 
        dk = d_model // n_heads

        # linear projections
        q = self.query(q) 
        k = self.key(k) 
        v = self.value(v) 

        # add number of heads
        q = q.view(B,T,n_heads,dk).permute(0,2,1,3)   # B,T,h,dk
        k = k.view(B,T,n_heads,dk).permute(0,2,1,3)  
        v = v.view(B,T,n_heads,dk).permute(0,2,1,3)  
        
        # attention 

        x = q @ k.transpose(-2,-1) # B,h,T,dk @ B,h,dk,T --> B,h,T,T
        x = x * dk ** -0.5 # B,h,T,T
        x = x.masked_fill(self.mask, float('-inf')) # B,h,T,T
        x = F.softmax(x, dim=(-1)) # B,n_h,T,T 
        x = x @ v  # B,h,T,T @ B,T,h,dv --> B,h,T,dv
        B,h,T,dv = x.shape
        x = x.transpose(2,1).contiguous().view(B,T,h*dv) #B,T,C
        out = self.att_proj(x) # B,T,C

        return out

In [16]:
class Model(nn.Module):

    def __init__(self, vocab_size, block_size, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.block_size = block_size
        self.embedding_table = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(block_size, d_model)
        self.mha = MultiHeadAttention(n_heads, d_model)
        # self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_heads, batch_first=True) #PyTorch class for debugging
        self.out = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, x, targets=None):

        embeds = self.embedding_table(x)
        positions = self.pos_embedding(torch.arange(self.block_size, device=device))
        x = embeds + positions

        x = self.mha(x) # b,t,c
        logits = self.out(x) # b,t,vocab_size

        # # PyTorch mha
        # # Create a causal mask of shape (T, T)
        
        # mask = torch.triu(torch.ones(T, T), diagonal=1).bool().unsqueeze(0)
        # mask = mask.repeat(B * n_heads, 1, 1).to(device)
        # x, att_scores = self.mha(x, x, x, attn_mask=mask)
        # logits = self.out(x)

        if targets is not None:
            logits = logits.reshape(-1, logits.shape[-1])
            targets = targets.reshape(-1)
            loss = F.cross_entropy(input=logits, target=targets)
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        B, T = idx.shape
        if T < self.block_size:
            # pad the input with zeros if it's less than block_size
            idx = F.pad(idx, (0, self.block_size - T))
        for _ in range(max_new_tokens):
            # use only the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [17]:
data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

print(f'Vocab: {vocab[:10]}.')
print(f'Vocab size: {vocab_size}')

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

print(decode(encode(data[1100:1150])))
data = torch.tensor(encode(data))

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

print(f'Training samples: {train.shape[0]}')
print(f'Validation samples: {val.shape[0]}')

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Xb, Yb = make_batches('train')

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3'].
Vocab size: 65
very dog to the commonalty.

Second Citizen:
Consi
Training samples: 1003854
Validation samples: 111539


In [18]:
m = Model(vocab_size, block_size).to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
print(m)
print(f'Total parameters: {n_params}')

@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

Model(
  (embedding_table): Embedding(65, 12)
  (pos_embedding): Embedding(6, 12)
  (mha): MultiHeadAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (query): Linear(in_features=12, out_features=12, bias=False)
    (key): Linear(in_features=12, out_features=12, bias=False)
    (value): Linear(in_features=12, out_features=12, bias=False)
    (att_proj): Linear(in_features=12, out_features=12, bias=False)
  )
  (out): Linear(in_features=12, out_features=65, bias=False)
)
Total parameters: 2208


In [19]:
# Training
for epoch in range(epochs):

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    #break
    if epoch % 1000 == 999:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")


Iteration 999. Training Loss: 2.844. Evaluation Loss: 2.826
Iteration 1999. Training Loss: 2.621. Evaluation Loss: 2.620
Iteration 2999. Training Loss: 2.570. Evaluation Loss: 2.579
Iteration 3999. Training Loss: 2.536. Evaluation Loss: 2.534
Iteration 4999. Training Loss: 2.499. Evaluation Loss: 2.525
Iteration 5999. Training Loss: 2.500. Evaluation Loss: 2.506
Iteration 6999. Training Loss: 2.496. Evaluation Loss: 2.500
Iteration 7999. Training Loss: 2.484. Evaluation Loss: 2.477
Iteration 8999. Training Loss: 2.474. Evaluation Loss: 2.461
Iteration 9999. Training Loss: 2.481. Evaluation Loss: 2.469


In [32]:
max_new_tokens = 2_000
seed_text = "Once upon a time, "
seed_idx = torch.tensor([stoi[c] for c in seed_text], device=device).unsqueeze(0)
predictions = m.generate(seed_idx, max_new_tokens).to(device)
print(decode(predictions[0].tolist()))

Once upon a time, foulathe at bilghind:
dy be nekg INI
SERRDY:
LTI:
Aighad peardang;
Com wile arr yut Verer, fot'f feiincuen avarsuresthe?

Whellat.

FOF,
Aeet Yonurth oundt the towcay aut.

Nhineso, marlded the itisedy tur yofuceve heerl, torurt ye.

S:
I soit on bou, hanour, batacam portund we Ce wobtpom Ry mecovaclllg mith dreestw rou wator wouevey that at min, ofes ford be hatr nistee thles haarlom
hriver heysicseshe theey ny bsh hragene, horf sieirnd astold weaw thish gothth ghor?

Whaw mate lael themct, ouomed homathy okighagey theea.
PS, Eing thin vap ros pwerald yllghare pruer com;
py yererlee-yastint,
A'd astou py fir'go cornorae scith th ton ere nollen anldon grous. Mathll lled bit hae.

AEKEAROO
Wut ou thinclis;
Eou kencf Pod me you fons chs
Ad tecolederthe min ie de wig:
Sure proun'lmwod toth st, yunnd'jo theris hiters Evitornt wain kowu,
Feae-tinsy fhe ilr eth yor-ja. Vin? the the jeld bo, ly corld ofu seng urers yous pokcet fan thithl.

SOEVHSE
HMERUKORDTRpiyou ileledisir

In [21]:
# DEBUGGING

torch.manual_seed(0)
X = torch.randn(4,6,2,6)

q = X
k = X
v = X

d_k = q.size()[-1]
attn_logits = torch.matmul(q, k.transpose(-2, -1)) # B,T,h,dk @ B,T,dk,h > B,T,h,h
attn_logits_s = attn_logits * d_k ** -0.5
# if mask is not None:
#     attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
attention = F.softmax(attn_logits_s, dim=-1)
values = torch.matmul(attention, v)

##########################

att = (q.transpose(1,2) @ k.permute(0,2,3,1)) * d_k ** -0.5 # B,h,T,dk @ B,h,dk,T -> B,h,T,T
B,nh,T,_ = att.shape
att_soft = F.softmax(att.view(-1,T,T), dim=(-1)).view(B,nh,T,T) # B,h,T,T 
mvalues = att_soft @ v.permute(0, 2, 1, 3).contiguous()  # B,h,T,T @ B,h,T,dv --> B,h,T,dv
mvalues = mvalues.view(B,T,nh,T)

#########################

mask = torch.triu(torch.ones(2, 2), diagonal=1).bool()
a_t = q @ k.transpose(-2,-1) # B,T,h,h
a_t_s = a_t * d_k ** -0.5 # B,T,h,h
# x = a_t_s.masked_fill(mask, float('-inf')) # B,T,h,h
a = F.softmax(a_t_s, -1) # B,T,h,h
va = a @ v # B,T,h,h @ B,T,h,dk > B,T,h,dk

###########################
print(torch.allclose(mvalues, values, atol=1e-5))
print(torch.allclose(values, va, atol=1e-4))

False
True
